In [ ]:
# ▩ 보스톤 하우징 집값을 예측하는 신경망 만들기 (수치예측)
# k-fold 교차검정 진행한다.


In [2]:
# 1. 보스톤 하우징 데이터를 불러옵니다.
	
from tensorflow.keras.datasets.boston_housing import load_data
import numpy as np

(x_train, y_train), (x_test, y_test) = load_data( path = 'boston_housing.npz', test_split = 0.2, seed = 777 )

# 1.1 데이터 표준화

mean = np.mean( x_train, axis =0 )
std = np.std( x_train, axis = 0 )

x_train = ( x_train - mean ) / std
x_test = ( x_test - mean ) / std


print(x_train.shape)       # 404, 13
print(y_train.shape)       # 404, 3
print(x_test.shape)         # 102, 13
print(y_test.shape)         # 102,


(404, 13)
(404,)
(102, 13)
(102,)


In [3]:
# 2. k-폴드 교차검정 진행

from sklearn.model_selection import KFold
k = 3
kfold = KFold( n_splits = k , random_state = 777 ,shuffle = True )


In [4]:
# 3. 모델생성

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_model():
    model = Sequential()
    model.add( Dense( 64, activation = 'relu', input_shape = (13, ) ) )    # input_shape는 첫번째만, 컬럼수만큼
    model.add( Dense( 32, activation = 'relu' ) )
    model.add( Dense(1) )    # 출력층에는 분류가 아니라 softmax 함수가 필요없습니다.
					                   # 예측되는 집값이 그대로 출력되면 됩니다.

    # 4. 모델 설정
    model.compile( optimizer = 'adam', loss = 'mse', metrics = ['mae'] )  # mae 는 오차입니다.(학습과정에서 정확도가 아니라 오차를 보려고)
    return model


In [ ]:
# 5. 모델 훈련

mae_list = []       # k 개의 mae 값을 담을 비어있는 리스트를 생성한다.

for train_index, val_index in kfold.split(x_train):     # 훈련데이터와 검정데이터를 위한 행번호를
											                                 # 각각 train_index와 val_index 에 담는다.
    x_train_fold, x_val_fold = x_train[ train_index ], x_train[ val_index ]    # 훈련 데이터와 검정데이터
    y_train_fold, y_val_fold = y_train[ train_index ], y_train[ val_index ]    # 훈련 정답과 검정 정답

    model = get_model()
    model.fit( x_train_fold, y_train_fold, epochs = 300 , validation_data = ( x_val_fold, y_val_fold ) )


    # 6. 모델평가
    _ , test_mae = model.evaluate( x_test, y_test )
    mae_list.append(test_mae)

print(mae_list)       # ex) [2.069694995880127, 2.2150659561157227, 2.307401180267334]



In [ ]:

# ※ 설명 : mse 는 평균제곱 오차이고 mae 는 평균절대오차입니다.
# 	      	mse 는 제곱을 해주는것이고 mae 는 절대값을 구하는것입니다. 

# 평균제곱오차는 회귀에서 자주 사용되는 손실함수입니다.
# 일반적인 회귀지표는 평균절대오차(mae) 입니다.


In [ ]:
# ※ 설명 : 분류를 할떄는 minmax 정규화를 사용하고 수치예측을 할때는 표준화를 해서 성능을 높입니다.

In [ ]:
# 위의 3개의 mae 값이 서로 다른 이유는 학습 및 검증 데이터가 서로 다르기 때문입니다.
# 그리고 k-hold 교차검정으로 얻어낸 최종 mae 값을 출력하려면 위의 3개의 값의 평균값을 출력하면
# 됩니다.


In [ ]:
# ※ 설명 : 검증 데이터 사용이 위와 같이 수치예측할 때만 필요하고 분류할때는 필요하지 않나 ?

# 답 : 분류할 때도 필요합니다.

# 분류할 때는 뭘위해서 필요한건가 ?

# 	" 오버피팅이 과도하게 발생하는지 확인하기 위해 필요합니다"

# 신경망의 층과 뉴런의 갯수를 너무 많이 주게 되면 훈련 데이터에 대한 정확도는 높아질 수 있으나 오버피팅이 발생할 수 있습니다.